In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Descarga OHLCV de Bitget vía API pública (/api/v2/mix/market/candles).
Script inspirado en tu SCRIPT1, pero usando llamadas HTTP según la documentación que
me pasaste. Comentarios y prints en español.
"""

import os
import time
from time import sleep
from pathlib import Path
from joblib import Parallel, delayed
import requests
import pandas as pd

# -----------------------------
# CONFIGURACIÓN (ajusta según necesites)
# -----------------------------
TIMEFRAME     = "4h"                   # ejemplo: '1m','3m','5m','15m','30m','1h','4h','1d'...
DATA_FOLDER   = "crypto_bitget_api"
START_DATE    = "2023-01-01"
N_JOBS        = 1
LIMIT         = 200                    # default por llamada; según doc max 1000
RETRY_DELAY   = 2
MAX_RETRIES   = 3
MIN_RECORDS   = 200
PRODUCT_TYPE  = "usdt-futures"         # según doc: usar "usdt-futures" (ejemplo)
BASE_URL      = "https://api.bitget.com"
RATE_SLEEP    = 0.06                   # pausa entre peticiones (≈16 req/s < 20 req/s límite)

# -----------------------------
# Mapas / utilidades según la doc
# -----------------------------
# La doc ofrece límites máximos de histórico por granularity (días). Los pongo aquí:
MAX_HISTORY_DAYS_PER_GRANULARITY = {
    "1m": 31,    # ~1 mes
    "3m": 31,
    "5m": 31,
    "15m": 52,
    "30m": 62,
    "1H": 83,
    "2H": 120,
    "4H": 240,
    "6H": 360,
    "12H": None,
    "1D": None,
    "3D": None,
    "1W": None,
    "1M": None
}
# Nota: la doc también menciona "maximum time query range is 90 days" para start/end en algún lugar.
# Para evitar consultas demasiado grandes, usaremos siempre paginación por `limit` y avanzaremos por último timestamp.

def normalize_granularity(tf: str) -> str:
    """
    Normaliza TIMEFRAME a la forma de la API: '4h' -> '4H', '1d' -> '1D', '1m'->'1m' (minúscula para minutos).
    La API acepta ambas en muchos casos; usamos formatos vistos en la doc.
    """
    s = tf.strip()
    if s[-1].lower() == 'm' and s[:-1].isdigit():
        return s.lower()  # 1m,3m,5m,15m,30m
    if s[-1].lower() == 'h' and s[:-1].isdigit():
        return s[:-1] + "H"  # 1H,4H,...
    if s[-1].lower() == 'd' and s[:-1].isdigit():
        return s[:-1] + "D"  # 1D
    if s[-1].lower() == 'w' and s[:-1].isdigit():
        return s[:-1] + "W"
    # fallback: devolver tal cual
    return s

GRANULARITY = normalize_granularity(TIMEFRAME)

def timeframe_to_ms(tf: str) -> int:
    tf = tf.lower().strip()
    if tf.endswith("m"):
        return int(tf[:-1]) * 60 * 1000
    if tf.endswith("h"):
        return int(tf[:-1]) * 3600 * 1000
    if tf.endswith("d"):
        return int(tf[:-1]) * 86400 * 1000
    if tf.endswith("w"):
        return int(tf[:-1]) * 7 * 86400 * 1000
    # fallback: 1 minute
    return 60 * 1000

CANDLE_MS = timeframe_to_ms(GRANULARITY)

# -----------------------------
# Cliente público Bitget (candles)
# -----------------------------
class BitgetPublicAPI:
    def __init__(self, base_url=BASE_URL):
        self.base_url = base_url.rstrip("/")
        self.session = requests.Session()
        self.session.headers.update({
            "Accept": "application/json",
            "User-Agent": "bitget-py-public/1.0"
        })

    def _get(self, path, params=None, timeout=15):
        url = f"{self.base_url}{path}"
        try:
            res = self.session.get(url, params=params, timeout=timeout)
        except Exception as e:
            raise RuntimeError(f"Excepción request: {e}")
        if res.status_code != 200:
            # Levantamos con texto para depurar
            raise RuntimeError(f"HTTP {res.status_code}: {res.text}")
        try:
            return res.json()
        except Exception as e:
            raise RuntimeError(f"Error decodificando JSON: {e} - raw: {res.text}")

    def get_candles(self, symbol, granularity=GRANULARITY, limit=LIMIT, startTime=None, endTime=None):
        """
        Llama a /api/v2/mix/market/candles según la doc.
        startTime, endTime en milisegundos (int). Retorna DataFrame (posiblemente vacío).
        """
        path = "/api/v2/mix/market/candles"
        params = {
            "symbol": symbol,
            "granularity": granularity,
            "limit": int(limit),
            "productType": PRODUCT_TYPE
        }
        if startTime is not None:
            params["startTime"] = int(startTime)
        if endTime is not None:
            params["endTime"] = int(endTime)

        resp = self._get(path, params=params)
        # Respuesta esperada: {"code":"00000","msg":"success","data":[[...],[...]]}
        if isinstance(resp, dict):
            if resp.get("code") and resp.get("code") != "00000":
                raise RuntimeError(f"API error: {resp}")
            data = resp.get("data") if "data" in resp else None
        elif isinstance(resp, list):
            data = resp
        else:
            data = None

        if not data:
            return pd.DataFrame()

        rows = []
        for c in data:
            try:
                ts = int(c[0])
                # normalizar a ms si vienen en s
                if ts < 1_000_000_000_000:
                    ts *= 1000
                rows.append({
                    "timestamp": ts,
                    "open": float(c[1]),
                    "high": float(c[2]),
                    "low": float(c[3]),
                    "close": float(c[4]),
                    "base_volume": float(c[5]) if len(c) > 5 and c[5] is not None else None,
                    "quote_volume": float(c[6]) if len(c) > 6 and c[6] is not None else None
                })
            except Exception:
                continue

        if not rows:
            return pd.DataFrame()
        df = pd.DataFrame(rows)
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
        df = df.sort_values("datetime").reset_index(drop=True)
        return df

    def get_futures_symbols(self):
        """Intento razonable para obtener contratos: /api/v2/mix/market/contracts (puede fallar)."""
        path = "/api/v2/mix/market/contracts"
        try:
            resp = self._get(path)
        except Exception as e:
            print(f"⚠️ get_futures_symbols HTTP error: {e}")
            return None

        data = None
        if isinstance(resp, dict) and "data" in resp:
            data = resp["data"]
        elif isinstance(resp, list):
            data = resp
        else:
            # intentar buscar lista dentro de resp
            if isinstance(resp, dict) and "data" in resp and isinstance(resp["data"], dict):
                for v in resp["data"].values():
                    if isinstance(v, list):
                        data = v
                        break

        if not data:
            return None

        symbols = []
        for item in data:
            if isinstance(item, str):
                symbols.append(item)
            elif isinstance(item, dict):
                for k in ("symbol", "instrumentId", "contractCode", "instId"):
                    if k in item and isinstance(item[k], str):
                        s = item[k].replace("-", "").replace("/", "")
                        symbols.append(s)
                        break
        return sorted(list(set(symbols)))


# -----------------------------
# Lógica principal (idéntica a SCRIPT1, adaptada a startTime/endTime)
# -----------------------------
api = BitgetPublicAPI()
os.makedirs(DATA_FOLDER, exist_ok=True)
start_run_time = time.time()

def get_first_candle(symbol, granularity=GRANULARITY, limit=LIMIT,
                     max_iterations=300, detect_api_ignoring_days=2):
    """
    Busca la primera vela disponible retrocediendo con endTime.
    Este método pedirá bloques con endTime y moverá endTime hacia atrás.
    """
    try:
        now_ts = int(pd.Timestamp.now(tz="UTC").timestamp() * 1000)
        step = limit * CANDLE_MS
        end_time = now_ts - step

        last_non_empty = None
        prev_block_min = None
        api_calls = 0

        for iteration in range(max_iterations):
            if api_calls >= 500:
                print(f"⚠️ {symbol}: máximo API calls alcanzado en get_first_candle, abortando.")
                break

            try:
                df = api.get_candles(symbol, granularity=granularity, limit=limit, endTime=end_time)
            except Exception as e:
                print(f"⚠️ {symbol}: error en get_candles durante get_first_candle: {e}")
                break
            api_calls += 1

            if df is None or df.empty:
                # si vacío cuando endTime muy antiguo, probablemente llegamos al inicio
                print(f"ℹ️ {symbol}: respuesta vacía buscando desde endTime={pd.to_datetime(end_time, unit='ms', utc=True)}")
                break

            block_min_ts = int(df['datetime'].min().timestamp() * 1000)
            block_max_ts = int(df['datetime'].max().timestamp() * 1000)

            last_non_empty = df

            # Detectar si la API devuelve solo data reciente y no avanza
            if (now_ts - block_max_ts) < detect_api_ignoring_days * 24 * 3600 * 1000:
                if prev_block_min is not None and block_min_ts >= prev_block_min:
                    print(f"⚠️ {symbol}: API parece devolver solo velas recientes (sin progreso). Abortando.")
                    return None

            if prev_block_min is not None and block_min_ts >= prev_block_min:
                print(f"⚠️ {symbol}: sin progreso buscando primer candle (block_min no disminuye). Parando.")
                break

            prev_block_min = block_min_ts
            # mover end_time hacia atrás
            end_time = block_min_ts - step
            sleep(RATE_SLEEP)

        if last_non_empty is not None:
            first_dt = last_non_empty['datetime'].min()
            print(f"✅ {symbol}: primer candle encontrado: {first_dt} (llamadas API: {api_calls})")
            return first_dt

    except Exception as e:
        print(f"⚠️ Error buscando primer candle para {symbol}: {e}")

    return None

def download_from_date(symbol, start_date=START_DATE, granularity=GRANULARITY, limit=LIMIT):
    """
    Descarga paginando desde start_date en adelante usando startTime y 'limit'.
    Avanza por last_timestamp + candle_ms.
    """
    all_blocks = []
    retries = 0
    since_ts = int(pd.Timestamp(start_date, tz="UTC").timestamp() * 1000)
    now_ts = int(pd.Timestamp.now(tz="UTC").timestamp() * 1000)
    first_attempt = True

    while since_ts < now_ts:
        try:
            req_dt = pd.to_datetime(since_ts, unit='ms', utc=True)
            print(f"⏳ {symbol}: solicitando desde {req_dt} (limit={limit})")
            try:
                df = api.get_candles(symbol, granularity=granularity, limit=limit, startTime=since_ts)
            except Exception as e:
                raise

            if df is not None and not df.empty:
                df['datetime'] = pd.to_datetime(df['datetime'], utc=True)

            if df is None or df.empty:
                # fallback: intentar sin startTime para obtener últimas velas
                if first_attempt:
                    print(f"⚠️ {symbol}: respuesta vacía desde {req_dt}, probando fallback sin startTime...")
                    try:
                        fallback = api.get_candles(symbol, granularity=granularity, limit=limit, startTime=None)
                    except Exception:
                        fallback = None

                    if fallback is not None and not fallback.empty:
                        fallback['datetime'] = pd.to_datetime(fallback['datetime'], utc=True)
                        block_min = fallback['datetime'].min()
                        block_max = fallback['datetime'].max()
                        # Si fallback devuelve solo velas recientes -> buscar primer real
                        if (pd.Timestamp.now(tz="UTC") - block_max) < pd.Timedelta("2d"):
                            print(f"⚠️ {symbol}: fallback devolvió solo velas recientes, buscando primer candle real...")
                            first_candle = get_first_candle(symbol, granularity, limit)
                            if first_candle:
                                since_ts = int(first_candle.timestamp() * 1000)
                                print(f"⚠️ {symbol}: primer candle real: {first_candle}")
                                first_attempt = False
                                continue
                        else:
                            since_ts = int(block_min.timestamp() * 1000)
                            print(f"⚠️ {symbol}: fallback aceptado, bloque desde {block_min}")
                            first_attempt = False
                            continue

                    # último recurso: buscar primer candle
                    first_candle = get_first_candle(symbol, granularity, limit)
                    if first_candle:
                        since_ts = int(first_candle.timestamp() * 1000)
                        print(f"⚠️ {symbol}: usando primer candle disponible: {first_candle}")
                        first_attempt = False
                        continue

                print(f"⏹️ {symbol}: fin de datos disponibles (respuesta vacía).")
                break

            all_blocks.append(df)
            last_ts = int(df['datetime'].max().timestamp() * 1000)
            # avanzar al siguiente bloque (última vela + 1 intervalo)
            since_ts = last_ts + CANDLE_MS

            first_attempt = False
            retries = 0

            if since_ts >= now_ts:
                break

            sleep(RATE_SLEEP)

        except Exception as e:
            retries += 1
            print(f"❌ {symbol}: Error descargando intento {retries}/{MAX_RETRIES}: {e}")
            if retries >= MAX_RETRIES:
                print(f"❌ {symbol}: máximo reintentos alcanzado, abortando descarga.")
                break
            sleep(RETRY_DELAY)

    if all_blocks:
        full_df = pd.concat(all_blocks).sort_values('datetime')
        full_df = full_df.drop_duplicates(subset='datetime')
        full_df.set_index('datetime', inplace=True)
        return full_df
    else:
        return None

def download_and_save(symbol):
    try:
        df = download_from_date(symbol)
        if df is None or df.empty:
            print(f"⚠️ No hay datos para {symbol}, omitiendo.")
            return False

        if len(df) < MIN_RECORDS:
            print(f"⚠️ {symbol} tiene solo {len(df)} registros (<{MIN_RECORDS}), se omite guardado.")
            return False

        # eliminar últimas 2 filas (como en tu SCRIPT1)
        if len(df) > 2:
            df_to_save = df.iloc[:-2]
        else:
            df_to_save = df

        df_to_save = df_to_save.copy()
        # quitar tz (Excel no soporta datetime tz-aware)
        if df_to_save.index.tz is not None:
            df_to_save.index = df_to_save.index.tz_localize(None)

        base_name = f"{symbol.replace('/', '')}_{GRANULARITY}"
        parquet_path = Path(DATA_FOLDER) / f"{base_name}.parquet"
        excel_path   = parquet_path.with_suffix(".xlsx")

        df_to_save.to_parquet(parquet_path, index=True)
        df_to_save.to_excel(excel_path)
        print(f"✅ Guardado {symbol} con {len(df_to_save)} registros (últimas 2 filas eliminadas).")
        return True

    except Exception as e:
        print(f"❌ Error guardando {symbol}: {e}")
        return False

# -----------------------------
# Obtener lista de símbolos (intento por API y fallback)
# -----------------------------
symbols = None
try:
    print("ℹ️ Intentando obtener lista de símbolos futuros desde la API de Bitget...")
    symbols = api.get_futures_symbols()
    if not symbols:
        raise RuntimeError("No se devolvieron símbolos desde la API.")
    print(f"✅ Símbolos obtenidos: {len(symbols)}")
except Exception as e:
    print(f"⚠️ No se pudo obtener la lista de símbolos desde la API: {e}")

# Fallback manual (edítala si quieres otros símbolos)
if not symbols:
    print("⚠️ Usando fallback de símbolos (edita la lista en el script si prefieres).")
    symbols = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "ADAUSDT"]

# -----------------------------
# Descarga en paralelo
# -----------------------------
results = Parallel(n_jobs=N_JOBS, backend="loky")(delayed(download_and_save)(sym) for sym in symbols)

# -----------------------------
# Resumen
# -----------------------------
success_count = sum(1 for r in results if r)
fail_count = len(results) - success_count
total = len(symbols)
print("\n================= RESUMEN =================")
print(f"Total símbolos: {total}")
print(f"Descargados correctamente: {success_count} ({(success_count/total*100):.2f}%)")
print(f"Descargas fallidas: {fail_count} ({(fail_count/total*100):.2f}%)")
print("==========================================")

end_run_time = time.time()
hours, remainder = divmod(end_run_time - start_run_time, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"\nTiempo total: {int(hours)} h {int(minutes)} min {int(seconds)} s")


ℹ️ Intentando obtener lista de símbolos futuros desde la API de Bitget...
⚠️ get_futures_symbols HTTP error: HTTP 400: {"code":"400172","msg":"Parameter verification failed","requestTime":1758373080558,"data":null}
⚠️ No se pudo obtener la lista de símbolos desde la API: No se devolvieron símbolos desde la API.
⚠️ Usando fallback de símbolos (edita la lista en el script si prefieres).
⏳ BTCUSDT: solicitando desde 2023-01-01 00:00:00+00:00 (limit=200)
⚠️ BTCUSDT: respuesta vacía desde 2023-01-01 00:00:00+00:00, probando fallback sin startTime...
⚠️ BTCUSDT: fallback devolvió solo velas recientes, buscando primer candle real...
ℹ️ BTCUSDT: respuesta vacía buscando desde endTime=2025-01-30 16:00:00+00:00
✅ BTCUSDT: primer candle encontrado: 2025-03-05 00:00:00+00:00 (llamadas API: 4)
⚠️ BTCUSDT: primer candle real: 2025-03-05 00:00:00+00:00
⏳ BTCUSDT: solicitando desde 2025-03-05 00:00:00+00:00 (limit=200)
⏳ BTCUSDT: solicitando desde 2025-06-03 04:00:00+00:00 (limit=200)
⏳ BTCUSDT: solic